In [1]:
import wandb
from wandb.integration.keras import WandbMetricsLogger # use this instead https://docs.wandb.ai/guides/integrations/keras
# from wandb.integration.keras import WandbCallback # for wandb-0.17.0
# from wandb.keras import WandbCallback # for wandb-0.16.3

wandb.login()
config={
    "learning_rate": 0.01,
    "epochs": 100,
    "batch_size": 32,
    "loss_function": "mean_squared_error",
    "architecture": "simple_dense"
}
wandb.init(project='keras_with_wandb', #entity='your_wandb_username', 
           config=config
           )

2024-06-06 03:47:55.133246: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-06 03:47:55.855295: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: Currently logged in as: ys7yoo. Use `wandb login --relogin` to force relogin


## Generate a dataset

In [2]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

# Generate synthetic data
X, y = make_regression(n_samples=1000, n_features=20, noise=0.1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Define a model

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

def build_model():
    model = Sequential([
        Dense(64, input_dim=20, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(learning_rate=wandb.config.learning_rate), loss=wandb.config.loss_function)
    return model

model = build_model()


/home/yyoo/tf2/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-06-06 03:47:59.612821: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:03:00.0, compute capability: 8.9


## Train the model

In [4]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test),
                    epochs=wandb.config.epochs, batch_size=wandb.config.batch_size,
                    callbacks=[WandbMetricsLogger()])
                    # callbacks=[WandbCallback()])

Epoch 1/100


I0000 00:00:1717613280.509877   47521 service.cc:145] XLA service 0x7fd3b40083e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717613280.509916   47521 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2024-06-06 03:48:00.541562: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-06 03:48:00.635281: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/25 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - loss: 38524.4609

I0000 00:00:1717613281.364484   47521 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
wandb: ERROR Unable to log learning rate.


25/25 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 37259.6289 - val_loss: 30288.5234
Epoch 2/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 20722.2910 - val_loss: 1939.0487
Epoch 3/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2156.1189 - val_loss: 633.4531
Epoch 4/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 369.4420 - val_loss: 237.2165
Epoch 5/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 197.2622 - val_loss: 176.9060
Epoch 6/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 151.1608 - val_loss: 157.1534
Epoch 7/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 133.3231 - val_loss: 159.2738
Epoch 8/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 139.7665 - val_loss: 119.0133
Epoch 9/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 95.4281 - val_loss: 108.1498
Epoch 10/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 71.5984 - val_loss: 103.1681
Epoch 11/100
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 67.8610 - val_loss: 83.9899
Epoch 12/100
25/25 ━━━━━━━

In [5]:
# Finish the wandb run
wandb.finish()

epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/epoch,99
epoch/loss,11.52234
epoch/val_loss,28.81441
